In [2]:
import numpy as np
from src.policy.jas_voc_policy import JAS_voc_policy
from src.utils.mouselab_jas import MouselabJas
from src.utils.data_classes import Action, MouselabConfig
from src.utils.distributions import Normal

%load_ext autoreload
%autoreload 2

In [3]:
# Number of projects:
num_projects = [(40+12+22)/3, 7, 50, 25, 27, 27, 10, 24, 4, 4, 37, (20+50+150+250)/4, 4,2,5,400,2,12, (10+30)/2, 6, (5+30)/2, (1000+200)/2, 3, (5+30)/2, 8, 6, 10,
    4, 5, 20, 14, (60+50)/2, 3, 18, 6, 37, 14, 6, (2+3)/2, 20, 20, 4, 20, 14, 5, 20, 20, 5, 45, 1500, 6, 5, 60, 10, 50, 6, 5, 100, 5, 4]
print("Mean", np.mean(num_projects))
print("Median", np.median(num_projects))

Mean 59.64444444444445
Median 13.0


In [13]:
papers_with_criteria = 66
num_papers = [3, 10, 5, 5, 9, 28, 41, 48, 9, 20, 3, 38, 38, 17, 21, 21, 7, 12, 16, 9, 4, 15, 40]
mean_criteria = np.sum(num_papers)/papers_with_criteria
print("Mean", mean_criteria)
top_criteria = np.argsort(num_papers)+1
print(list(top_criteria)[-10:][::-1])

utilized_in_half = [7,8,12,13,23]

Mean 6.348484848484849
[8, 7, 23, 13, 12, 6, 16, 15, 10, 14]


In [ ]:
electric_criteria = [13, 17, 12, 4]
it_criteria = [5, 7, 49, 15, 5]
pharmaceutical_criteria = [4]


In [8]:
# Domains
domains = ["Eletronic/Electricity and Chemical", "Textile", "Aerospacial", "Spacial", "Chemical", "Chemical", 
    "Eletronic/Electricity", "Industrial", "Non-Specified", "Non-Specified", "Metallurgy", "Eletronic/Electricity",
    "Non-Specified", "Non-Specified", "Biotechnology", "Automotive", "Information Technology", "Industrial", "Industrial"
    "Research", "Pharmaceutical", "Non-Specified", "Metallurgy", "Pharmaceutical", "Non-Specified", "Information Technology",
    "Information Technology", "Telecommunications", "Non-Specified", "Non-Specified", "Nuclear", "Pharmaceutical", "Non-Specified",
    "Non-Specified", "Industrial", "Non-Specified", "Metallurgy", "Eletronic/Electricity", "Non-Specified", "Government Sponsored"
    "Civil, Mechanical, and others", "Non-Specified", "Pharmaceutical", "Pharmaceutical", "Manufacturing", "Non-Specified",
    "Pharmaceutical", "Information Technology", "Civil, Mechanical, and others", "Pharmaceutical", "Non-Specified", "Eletronic/Electricity",
    "Agriculture, Innovation, Textile and others", "Private/Public Sector", "Non-Specified", "Non-Specified", "Eletronic/Electricity",
    "Government Sponsored", "Non-Specified", "Pharmaceutical", "Private/Public Sector", "Energy", "Military", "Innovation",
    "Electricity/Mechanical/Telecommunications/IT", "Non-Specified"]

from collections import Counter

counter = Counter(domains)
counter.most_common()


[('Non-Specified', 19),
 ('Pharmaceutical', 8),
 ('Eletronic/Electricity', 5),
 ('Information Technology', 4),
 ('Industrial', 3),
 ('Metallurgy', 3),
 ('Chemical', 2),
 ('Private/Public Sector', 2),
 ('Eletronic/Electricity and Chemical', 1),
 ('Textile', 1),
 ('Aerospacial', 1),
 ('Spacial', 1),
 ('Biotechnology', 1),
 ('Automotive', 1),
 ('IndustrialResearch', 1),
 ('Telecommunications', 1),
 ('Nuclear', 1),
 ('Government SponsoredCivil, Mechanical, and others', 1),
 ('Manufacturing', 1),
 ('Civil, Mechanical, and others', 1),
 ('Agriculture, Innovation, Textile and others', 1),
 ('Government Sponsored', 1),
 ('Energy', 1),
 ('Military', 1),
 ('Innovation', 1),
 ('Electricity/Mechanical/Telecommunications/IT', 1)]

In [7]:
def diff_rec(str1, str2):
    if (len(str1) == 0) and (len(str2) == 0):
        return 0
    elif len(str1) == 0:
        return sum([ord(c) for c in str2])
    elif len(str2) == 0: 
        return sum([ord(c) for c in str1])
    elif str1[0] == str2[0]:
        return diff_rec(str1[1:], str2[1:])
    else:
        diff1 = ord(str1[0]) + diff_rec(str1[1:], str2)
        diff2 = ord(str2[0]) + diff_rec(str1, str2[1:])
        return min(diff1, diff2)

print(diff_rec("through", "trouf"))

413


In [11]:
expert_costs = [1, 1]
expert_taus = [0.05, 0.01]
config = MouselabConfig()
tree = [[1, 2], [], []]
init = [Normal(0, 1), Normal(0, 20), Normal(0, 10)]
env = MouselabJas(tree, init, expert_costs, expert_taus, config)
policy = JAS_voc_policy()

In [12]:
policy.myopic_voc_normal(env, Action(0, 1))

7.786556010920246

In [13]:
policy.myopic_voc_normal(env, Action(0, 2))

3.641828101973597

In [14]:
policy.myopic_voc_normal(env, Action(1, 1))

7.136496464611085

In [15]:
policy.act(env)

Action(expert=0, query=1)

In [3]:
from src.utils.distributions import Categorical

def get_cat(probs):
    vals = [i+1 for i in range(len(probs))]
    dist = Categorical(vals, probs)
    return dist

In [ ]:
prob1 = [0.2, 0.4, 0.3, 0.05, 0.05]
expert_precision = [0.4, 0.3, 0.15, 0.1, 0.05]